<a href="https://colab.research.google.com/github/jesusvillota/CSS_DataScience_2025/blob/main/Session3/3_5_LLM_II_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="max-width: 880px; margin: 20px auto 22px; padding: 0px; border-radius: 18px; border: 1px solid #e5e7eb; background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%); box-shadow: 0 8px 26px rgba(0,0,0,0.06); overflow: hidden;">

  <!-- Banner Header -->
  <div style="padding: 34px 32px 14px; text-align: center; line-height: 1.38;">
    <div style="font-size: 13px; letter-spacing: 0.14em; text-transform: uppercase; color: #6b7280; font-weight: bold; margin-bottom: 5px;">
      Session #3
    </div>
    <div style="font-size: 29px; font-weight: 800; color: #14276c; margin-bottom: 4px;">
      LLMs
    </div>
    <div style="font-size: 26px; font-weight: 800; color: #14276c; margin-bottom: 4px;">
      Part III: Function Calling
    </div>
    <div style="font-size: 16.5px; color: #374151; font-style: italic; margin-bottom: 0;">
      Data Science for Economics: Mastering Unstructured Data
    </div>
  </div>

  <!-- Logo Section -->
  <div style="background: none; text-align: center; margin: 30px 0 10px;">
    <img src="https://www.cemfi.es/images/Logo-Azul.png" alt="CEMFI Logo" style="width: 158px; filter: drop-shadow(0 2px 12px rgba(56,84,156,0.05)); margin-bottom: 0;">
  </div>

  <!-- Name -->
  <div style="font-family: 'Times New Roman', Times, serif; color: #38549c; text-align: center; font-size: 1.22em; font-weight: bold; margin-bottom: 0px;">
    Jesus Villota Miranda © 2025
  </div>

  <!-- Contact info -->
  <div style="font-family: 'Times New Roman', Times, serif; color: #38549c; text-align: center; font-size: 1em; margin-top: 7px; margin-bottom: 20px;">
    <a href="mailto:jesus.villota@cemfi.edu.es" style="color: #38549c; text-decoration: none; margin-right:8px;" title="Email">
      <!-- <img src="https://cdn-icons-png.flaticon.com/512/11679/11679732.png" alt="Email" style="width:18px; vertical-align:middle; margin-right:5px;"> -->
      jesus.villota@cemfi.edu.es
    </a>
    <span style="color:#9fa7bd;">|</span>
    <a href="https://www.linkedin.com/in/jesusvillotamiranda/" target="_blank" style="color: #38549c; text-decoration: none; margin-left:7px;" title="LinkedIn">
      <!-- <img src="https://1.bp.blogspot.com/-onvhHUdW1Us/YI52e9j4eKI/AAAAAAAAE4c/6s9wzOpIDYcAo4YmTX1Qg51OlwMFmilFACLcBGAsYHQ/s1600/Logo%2BLinkedin.png" alt="LinkedIn" style="width:17px; vertical-align:middle; margin-right:5px;"> -->
      LinkedIn
    </a>
  </div>
</div>

This notebook replicates the methodology in [CEMFI Working Paper 2501](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5006857).

In [5]:
! pip install load_dotenv groq

In [ ]:
import os
import json
import pandas as pd

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv('GROQ_API_KEY')

In [ ]:
from groq import Groq
client = Groq(api_key=api_key)
MODEL = 'llama3-70b-8192'

In [12]:
def news_parser(firms):
    response = []
    for firm in firms:
        response.append({
            "firm": firm["firm"],
            "ticker": firm.get("ticker", ""),
            "shock_type": firm.get("shock_type", ""),
            "shock_magnitude": firm.get("shock_magnitude", ""),
            "shock_direction": firm.get("shock_direction", ""),
        })
    return response

def run_conversation(user_prompt):
    # Step 1: send the conversation and available functions to the model
    messages = [
        {
            "role": "system",
            "content":  f"""
                            You are a function calling LLM that analyses business news in Spanish. 
                        """
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "news_parser",
                "description": f"""
                                    For every article, you must identify the firms directly affected by the news. Do not include every firm mentioned in the article, only include those that are directly affected by the shocks narrated therein. 
                                    The identified firms must be Spanish and should be publicly listed in the Spanish exchange (their ticker is of the form 'TICKER.MC'). Do not include non-Spanish foreign firms. Do not include Spanish firms that are not publicly traded.
                                    For each identified firm, classify the shocks that affect them (type, magnitude, category). The type of shock can be 'demand', 'supply', 'financial', 'policy', or 'technology'. The magnitude can be 'minor' or 'major'. The direction can be 'positive' or 'negative'.
                                    If a firm is neutral to the article, do NOT include it in the analysis.
                                """,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "firms": {
                            "type": "array",
                            "description": f"""
                                                List the Spanish firms impacted by the reported news. Such firms must be publicly listed in the Spanish stock exchange and have a stock market ticker of the form TICKER.MC. 
                                                Foreign firms (not listed in the Spanish exchange and whose ticker is not TICKER.MC) are not to be included here. Do not include firms that are mentioned just for contextual comparison but are not directly affected by the events described in the article.
                                                If a firm is neutral to the article, do not include it in the list.
                                                Some times the article mentions explicitly the Spanish ticker of those firms that are directly affected (and hence, the firms to include here). e.g: Iberdrola (IBE.MC).
                                            """,
                            "items": {
                                "type": "object",
                                "properties": {
                                    "firm": {
                                        "type": "string",
                                        "description": "State the Spanish firm (within the list 'firms') in which you will focus the analysis. This firm should be publicly traded in the Spanish exchange with a ticker of the form 'TICKER.MC'. ",
                                    },
                                    "ticker": {
                                        "type": "string",
                                        "description": "Specify its stock market ticker of the Spanish firm in Yahoo Finance format (note that Spanish firms' tickers end with '.MC', e.g., ITX.MC for Inditex, ACX.MC for Acerinox, SAN.MC for Banco Santander, NTGY.MC for Naturgy).",
                                    },
                                    "shock_type": {
                                        "type": "string",
                                        "enum": ["demand", "supply", "financial", "policy", "technology"],
                                        "description": "Classify the type of shock implied by the news article. Choose 'demand' for events impacting consumer demand, 'supply' for events affecting the supply of goods or services, 'financial' for events related to financial markets or conditions, 'policy' for events stemming from changes in government policies or regulations, and 'technology' for events resulting from significant technological advancements or disruptions.",
                                    },
                                    "shock_magnitude": {
                                        "type": "string",
                                        "enum": ["minor", "major"],
                                        "description": "How strong do you expect the shock to be: 'minor' or 'major'?",
                                    },
                                    "shock_direction": {
                                        "type": "string",
                                        "enum": ["positive", "negative"],
                                        "description": f"""
                                                        In what direction do you expect the shock to affect this firm? Choose one of the available options: 'positive' or 'negative'.
                                                        Choose 'positive' for beneficial impacts and 'negative' for adverse impacts.
                                                        Do not state 'neutral' here. If the firm is neutral to the article, do not include it in the list of firms.
                                                        """,
                                    },
                                },
                                "required": ["firm"],
                            },
                        },
                    },
                    "required": ["firms"],
                },
            },
        },
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        
        # Step 3: call the function
        available_functions = {
            "news_parser": news_parser,
        }
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                firms=function_args.get("firms")
            )
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": json.dumps(function_response),
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )  # get a new response from the model where it can see the function response
        
        return second_response.choices[0].message.content, function_response


user_prompt = f"""
Cellnex tendrá más competencia en Europa.  La filial de Telefónica (TEF.MC) Telxius Telecom ha acordado vender su división de torres de telecomunicaciones en Europa y Latinoamérica a American Tower (AMT), lo cual aumentará la presencia de ésta en Europa e incrementará la competencia para el grupo español de telecomunicaciones inalámbricas Cellnex Telecom (CLNX.MC), señala Equita Sim. La transacción "supone la entrada de un nuevo operador independiente de torres en el mercado español y potencialmente más competencia para el crecimiento futuro también en el mercado europeo", sostiene la correduría. Cellnex llegó a un acuerdo en noviembre con CK Hutchison (0001.HK) para comprar el negocio europeo de torres y sus activos del conglomerado cotizado en Hong Kong. La acción de Telefónica sube un 9,6% a EUR3,94 y la de Cellnex avanza un 0,3% a EUR47,79.
"""

completion_text, structured_output = run_conversation(user_prompt)
print("Completion Text:", completion_text)
print("Structured Output:", structured_output)


Completion Text: The function has parsed the news article and identified the impact of the event on the mentioned companies.

Here's a breakdown of the output:

* For Cellnex Telecom (CLNX.MC):
	+ Shock type: supply (the increased competition from American Tower will affect Cellnex's supply chain or operations)
	+ Shock magnitude: minor (the impact is expected to be relatively small)
	+ Shock direction: negative (the event is expected to have a negative impact on Cellnex's business)
* For Telefónica (TEF.MC):
	+ Shock type: financial (the sale of Telxius Telecom's tower division to American Tower will have a financial impact on Telefónica)
	+ Shock magnitude: minor (the impact is expected to be relatively small)
	+ Shock direction: positive (the event is expected to have a positive impact on Telefónica's financials)

This output can be used to inform investment decisions, risk assessments, or other business-related activities.
Structured Output: [{'firm': 'Cellnex Telecom', 'ticker': '

In [13]:
structured_output

[{'firm': 'Cellnex Telecom',
  'ticker': 'CLNX.MC',
  'shock_type': 'supply',
  'shock_magnitude': 'minor',
  'shock_direction': 'negative'},
 {'firm': 'Telefónica',
  'ticker': 'TEF.MC',
  'shock_type': 'financial',
  'shock_magnitude': 'minor',
  'shock_direction': 'positive'}]